In [1]:
import ipywidgets as widgets
from IPython.display import display
record_button= widgets.Button(
description="Record",
disabled=False,
    button_style="success",
    icon="microphone"
)

display(record_button)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

In [2]:

import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
    
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

In [3]:

import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (Cirrus Logic Superi', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Headset (E)', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputC

In [4]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2,
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [5]:
%pip install vosk


Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable
  Using cached vosk-0.3.45-py3-none-win_amd64.whl (14.0 MB)
  Using cached srt-3.5.3-py3-none-any.whl
  Using cached websockets-11.0.3-cp311-cp311-win_amd64.whl (124 kB)



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
%pip install transformers


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

                                              0.0/977.5 kB ? eta -:--:--
     -                                       30.7/977.5 kB 1.3 MB/s eta 0:00:01
     -                                       30.7/977.5 kB 1.3 MB/s eta 0:00:01
     ---                                   81.9/977.5 kB 657.6 kB/s eta 0:00:02
     ---                                   92.2/977.5 kB 525.1 kB/s eta 0:00:02
     ------                               163.8/977.5 kB 756.6 kB/s eta 0:00:02
     --------                             225.3/977.5 kB 860.2 kB/s eta 0:00:01
     ---------                            256.0/977.5 kB 874.6 kB/s eta 0:00:01
     ---------------                        389.1/977.5 kB 1.1 MB/s eta 0:00:01
     -------------------                    491.5/977.5 kB 1.3 MB/s eta 0:00:01
     -------------------------              665.6/977.5 kB 1.4 

In [ ]:
%pip install torch 

In [12]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    while not messages.empty():
        frames=recordings.get()
        
        rec.AcceptWaveform(b''join(frames))
        result=rec.Result()
        text=json.loads(result)["text"]
        
    cased=subprocess.check_output("python recasepunc.py predict recasepunc/checkpoint", shell=True, text=True, input=text) 
    ouput.append_stdput(cased)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (443433387.py, line 13)

In [ ]:
from vosk import Model
Model(model_name="vosk-model-small-en-us-0.15")

In [ ]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
    
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(cased)
        time.sleep(1)